In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import catboost as cb
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

X_train = pd.read_csv("train/X_train.csv").drop(columns="dt_iso")
y_train = pd.read_csv("train/y_train.csv")

X_val = pd.read_csv("validation/X_val.csv").drop(columns="dt_iso")
y_val = pd.read_csv("validation/y_val.csv")

X_test = pd.read_csv("test/X_test.csv").drop(columns="dt_iso")
y_test = pd.read_csv("test/y_test.csv")

In [11]:
# Plot data
X_train


,clouds_all_ Barcelona,clouds_all_Bilbao,clouds_all_Madrid,clouds_all_Seville,clouds_all_Valencia,humidity_ Barcelona,humidity_Bilbao,humidity_Madrid,humidity_Seville,humidity_Valencia,...,generation other,generation other renewable,generation solar,generation waste,generation wind onshore,forecast solar day ahead,forecast wind onshore day ahead,total load forecast,price day ahead,price actual
0,-0.500,-0.5,-0.5,-0.500,-0.5,0.823529,0.735294,-0.264706,0.088235,0.147059,...,-1.210137,-0.703922,-0.226415,-1.185714,0.342575,-0.217764,0.361454,-0.428976,0.015436,0.529854
1,-0.500,-0.5,-0.5,-0.500,-0.5,0.823529,0.735294,-0.264706,0.088235,0.147059,...,-1.210137,-0.842694,-0.226016,-1.200000,0.233238,-0.218153,0.228852,-0.643094,-0.121375,0.495356
2,-0.500,-0.5,-0.5,-0.500,-0.5,0.823529,0.735294,-0.235294,-0.029412,0.176471,...,-1.210137,-0.703922,-0.226016,-1.185714,0.137120,-0.221270,0.136946,-0.994241,-0.174047,0.464379
3,-0.500,-0.5,-0.5,-0.500,-0.5,0.823529,0.735294,-0.235294,-0.029412,0.176471,...,-1.210137,-0.582009,-0.226016,-1.257143,0.087156,-0.223607,0.067673,-1.296160,-0.520179,0.101098
4,-0.500,-0.5,-0.5,-0.500,-0.5,0.823529,0.735294,-0.235294,-0.029412,0.176471,...,-1.210137,-0.644086,-0.229210,-1.285714,0.019268,-0.220880,0.001372,-1.633043,-0.784225,-0.129825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24539,1.375,-0.5,0.5,0.000,0.0,0.617647,0.000000,0.441176,0.147059,-0.794118,...,0.472407,1.865852,0.755516,0.214286,-0.205008,0.696143,-0.222222,0.650762,0.869138,0.865677
24540,1.750,0.0,0.5,0.000,0.0,0.823529,-0.117647,0.617647,0.147059,-0.882353,...,0.308427,1.994971,0.814615,0.200000,-0.018148,0.791975,-0.065615,0.630177,0.763793,0.784009
24541,1.375,0.0,0.5,1.375,0.0,0.470588,-0.470588,-0.500000,-0.235294,-1.029412,...,0.208505,1.738731,0.772686,0.214286,0.249146,0.790806,0.201417,0.367065,0.687179,0.672772
24542,1.375,0.0,0.5,0.000,0.5,0.470588,-0.470588,-0.500000,-0.235294,-1.205882,...,0.208505,1.865852,0.694819,0.300000,0.487089,0.702766,0.424326,0.221971,0.449812,0.352437


In [ ]:
degrees = [1, 2, 3, 4]
validation_errors = []
best_degree=1

"""for degree in degrees:
    # Create polynomial features
    poly = PolynomialFeatures(degree=degree)
    X_train_poly = poly.fit_transform(X_train)
    X_val_poly = poly.transform(X_val)
    
    # Train a linear regression model on the polynomial features
    model = LinearRegression()
    model.fit(X_train_poly, y_train)
    
    # Predict on the validation set and calculate error
    y_val_pred = model.predict(X_val_poly)
    val_mse = mean_squared_error(y_val, y_val_pred)
    validation_errors.append(val_mse)
    
    print(f"Degree: {degree}, Validation MSE: {val_mse}")

# Select the best polynomial degree based on the lowest validation error
best_degree = degrees[np.argmin(validation_errors)]
print(f"Best Polynomial Degree: {best_degree}")"""

# Combine training and validation sets
X_train_val = pd.concat([X_train, X_val])
y_train_val = pd.concat([y_train, y_val])

# Create polynomial features with the best degree
poly = PolynomialFeatures(degree=best_degree)
X_train_val_poly = poly.fit_transform(X_train_val)
X_test_poly = poly.transform(X_test)

# Train the model on the combined training and validation set
final_model = LinearRegression()
final_model.fit(X_train_val_poly, y_train_val)

# Predict on the test set
y_test_pred = final_model.predict(X_test_poly)

# Calculate the Mean Squared Error
test_mse = mean_squared_error(y_test, y_test_pred)
print(f"Test Mean Squared Error: {test_mse}")

# Plot actual vs predicted values for the test set
plt.plot(range(len(y_test)), y_test, label="Actual Values")
plt.plot(range(len(y_test_pred)), y_test_pred, label="Predicted Values")
plt.legend()
plt.xlabel("Time Steps (Test Data)")
plt.ylabel("Value")
plt.show()


In [ ]:
#neural network
model = MLPRegressor(hidden_layer_sizes=(10,), max_iter=2000, random_state=0)

# Train the model on the training data
model.fit(X_train, y_train)
# Predict on the test set
y_pred = model.predict(X_test)

# Calculate the Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Plot actual vs predicted values for the test set
plt.plot(range(len(y_test)), y_test, label="Actual Values")
plt.plot(range(len(y_pred)), y_pred, label="Predicted Values")
plt.legend()
plt.xlabel("Time Steps (Test Data)")
plt.ylabel("Value")
plt.show()

### Decision Tree

In [ ]:
"""
catboost_model = cb.CatBoostRegressor(verbose=0, random_state=42)
catboost_model.fit(X_train, y_train, eval_set=(X_val, y_val))

lightgbm_model = lgb.LGBMRegressor(verbose=-1, random_state=42)
lightgbm_model.fit(X_train, y_train, eval_set=(X_val, y_val))

xgboost_model = xgb.XGBRegressor(random_state=42)
xgboost_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)

y_pred_catboost = catboost_model.predict(X_test)
mse_catboost = mean_squared_error(y_test, y_pred_catboost)
print(f"Mean Squared Error Catboost: {mse_catboost}")

y_pred_lightgbm = lightgbm_model.predict(X_test)
mse_lightgbm = mean_squared_error(y_test, y_pred_lightgbm)
print(f"Mean Squared Error LightGBM: {mse_lightgbm}")

y_pred_xgboost = xgboost_model.predict(X_test)
mse_xgboost = mean_squared_error(y_test, y_pred_xgboost)
print(f"Mean Squared Error XGBoost: {mse_xgboost}")

y_pres_ensamble = (y_pred_catboost + y_pred_lightgbm + y_pred_xgboost) / 3
mse_ensamble = mean_squared_error(y_test, y_pres_ensamble)
print(f"Mean Squared Error Ensamble: {mse_ensamble}")

plt.plot(range(len(y_test)), y_test, label="Actual Values")
plt.plot(range(len(y_pred)), y_pred, label="Predicted Values")
plt.legend()
plt.title("Gradient Boosting decision tree (Ensamble of 3 models)")
plt.xlabel("Time Steps (Test Data)")
plt.ylabel("Value")
plt.show()
"""

# Decision tree regressor
model_dt = DecisionTreeRegressor(random_state=42)
model_dt.fit(X_train, y_train)
y_pred_dt = model_dt.predict(X_test)
mse_dt = mean_squared_error(y_test, y_pred_dt)
print(f"Mean Squared Error Decision Tree: {mse_dt}")

plt.plot(range(len(y_test)), y_test, label="Actual Values")
plt.plot(range(len(y_pred_dt)), y_pred_dt, label="Predicted Values")
plt.legend()
plt.title("Decision Tree Regressor")
plt.xlabel("Time Steps (Test Data)")
plt.ylabel("Value")
plt.show()

### Random forrest

In [ ]:
model_rf = RandomForestRegressor(random_state=42)
model_rf.fit(X_train, y_train, )
y_pred_rf = model_rf.predict(X_test)
mse_rf = mean_squared_error(y_test, y_pred_rf)
print(f"Mean Squared Error Random Forest: {mse_rf}")

plt.plot(range(len(y_test)), y_test, label="Actual Values")
plt.plot(range(len(y_pred_rf)), y_pred_rf, label="Predicted Values")
plt.legend()
plt.title("Random Forest")
plt.xlabel("Time Steps (Test Data)")
plt.ylabel("Value")
plt.show()

### SVM with kernel

In [ ]:
# SVM regressor (Support Vector Machine)
from sklearn.svm import SVR
model_svm = SVR()
model_svm.fit(X_train, y_train)
y_pred_svm = model_svm.predict(X_test)
mse_svm = mean_squared_error(y_test, y_pred_svm)
print(f"Mean Squared Error SVM: {mse_svm}")

plt.plot(range(len(y_test)), y_test, label="Actual Values")
plt.plot(range(len(y_pred_svm)), y_pred_svm, label="Predicted Values")
plt.legend()
plt.title("Support Vector Machine")
plt.xlabel("Time Steps (Test Data)")
plt.ylabel("Value")
plt.show()

### Additive model